In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs


In [22]:
stats = {"average-1st-half-margin": "",
             "average-2nd-half-margin": "",
             "three-point-pct": "",
             "opponent-three-point-pct": "",
             "points-per-game": "",
             "opponent-points-per-game": ""
            }
    

In [46]:
def multiindex_off_suffix(df):
    columns = df.columns
    header = [col.split("__")[1] for col in columns]
    new_indices = zip(header, [col.split("__")[0] for col in columns])
    new_df = df.copy()
    new_df.columns = pd.MultiIndex.from_tuples(list(new_indices))
    return new_df

def scrape_ncaa_data(stats):
    base_url = "https://www.teamrankings.com/ncaa-basketball/stat/"
    full_df = pd.DataFrame()
    for stat in stats.keys():
        url = base_url + stat
        try:
            dfs = pd.read_html(url)
            df = dfs[0].set_index("Team")
            df.columns = [f"{col}__{stat}" for col in df.columns]
        except:
            print(f"Failed for {stat}")
        if len(full_df) > 0:
            full_df = pd.merge(df, full_df, left_index=True, right_index=True, suffixes = (None, None))
        else:
            full_df = df

    return multiindex_off_suffix(full_df)

In [47]:
test_df = scrape_ncaa_data(stats)

In [48]:
test_df.head()

opponent-points-per-game                                        \
                                Rank  2022 Last 3 Last 1  Home  Away  2021   
Team                                                                         
Tennessee                          1  55.1   59.7   67.0  51.1  59.1  62.9   
Houston                            2  55.2   67.0   61.0  53.4  57.9  58.8   
North Texas                        3  55.5   51.0   42.0  55.3  55.7  56.5   
St Marys                           4  56.9   57.7   59.0  55.9  58.4  61.3   
Rutgers                            5  58.5   64.3   55.0  52.8  70.9  66.5   

            points-per-game               ... average-2nd-half-margin       \
                       Rank  2022 Last 3  ...                    Home Away   
Team                                      ...                                
Tennessee               102  73.5   68.7  ...                    14.0  2.7   
Houston                  65  75.2   75.7  ...                     8.6  6.9   
North Texas             347  61.5   59.3  ...                     0.1  4.8   
St Marys                172  70.7   67.3  ...                     6.4  5.7   
Rutgers                 151  71.4   79.0  ...                    10.9 -2.3   

                  average-1st-half-margin                                       
             2021                    Rank  2022 Last 3 Last 1  Home Away  2021  
Team                                                                            
Tennessee    +4.8                       3   9.7    6.3   -6.0  13.8  5.6  +5.7  
Houston      +6.0                       1  12.1   -0.7    1.0  14.2  8.8  +9.8  
North Texas  +1.8                      86   3.0    7.3    4.0   6.6  0.8  +5.8  
St Marys     +4.2                      10   7.9    5.3    9.0  11.3  2.6  +4.0  
Rutgers      +2.8                      27   6.0    2.3   10.0   9.9 -2.6  -0.8  

[5 rows x 42 columns]

In [50]:
test_df.to_csv("ncaa_stats_20230203.csv")